In [125]:
import numpy as np
import os

class SimpleDatasetLoader:
    def init(self):
        return

    def load(self, imagePaths, verbose=-1):
        # initialize the list of features and labels
        data = []
        labels = []
        # loop over the input images
        for (i, imagePath) in enumerate(imagePaths):
            # load the image and extract the class label assuming
            # that our path has the following format:
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = f'{imagePath.split(os.path.sep)[-3]}_{imagePath.split(os.path.sep)[-2]}'

            data.append(image)
            labels.append(label)

            # show an update every verbose images
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1,
                                                      len(imagePaths)))
                # return a tuple of the data and labels
        return (np.array(data), np.array(labels))


In [126]:
import cv2
import json
import pandas as pd
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
    
class Parametros:
  dataset = "./images"
  neighbors = 1
  jobs = -1

args = vars(Parametros)

print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
sdl = SimpleDatasetLoader()
(data, labels) = sdl.load(imagePaths, verbose=500)

sift_df = pd.DataFrame()

for i in range(len(data)):
    image = data[i]
    label = labels[i]

    # GENERATING :SIFT: METRICS
    sift = cv2.SIFT_create()

    keypoints, descriptors = sift.detectAndCompute(image, None)
    author = label

    # CONVERTING KEYPOINTS TO COORDINATES
    keypoint_list = list(keypoints)
    descriptors_list = descriptors.tolist()


    cols = list(range(len(descriptors_list[0])))
    cols.insert(0, 'class_name')

    for descriptors in descriptors_list: 
        descriptors.insert(0, author)

    sift_matrix = list(descriptors_list)

    df = pd.DataFrame(sift_matrix, columns=cols)

    sift_df = pd.concat([sift_df, df])

[INFO] loading images...


In [127]:
sift_df

,class_name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,monet_ai_signatures,31.0,13.0,6.0,5.0,2.0,12.0,21.0,21.0,14.0,...,3.0,1.0,2.0,18.0,16.0,12.0,22.0,45.0,7.0,1.0
1,monet_ai_signatures,5.0,141.0,116.0,8.0,1.0,1.0,0.0,0.0,19.0,...,0.0,7.0,5.0,3.0,5.0,11.0,5.0,1.0,1.0,7.0
2,monet_ai_signatures,71.0,35.0,5.0,9.0,78.0,60.0,3.0,6.0,137.0,...,29.0,27.0,2.0,0.0,0.0,2.0,3.0,5.0,23.0,11.0
3,monet_ai_signatures,11.0,39.0,96.0,25.0,1.0,0.0,0.0,2.0,59.0,...,0.0,1.0,8.0,28.0,36.0,6.0,0.0,0.0,0.0,0.0
4,monet_ai_signatures,12.0,4.0,4.0,2.0,1.0,6.0,12.0,8.0,0.0,...,24.0,20.0,43.0,47.0,56.0,77.0,8.0,2.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,monet_original_signatures,16.0,112.0,31.0,1.0,0.0,0.0,0.0,0.0,10.0,...,1.0,7.0,3.0,30.0,84.0,21.0,15.0,6.0,8.0,5.0
455,monet_original_signatures,3.0,1.0,7.0,6.0,7.0,39.0,123.0,19.0,38.0,...,8.0,0.0,5.0,28.0,10.0,46.0,19.0,6.0,0.0,0.0
456,monet_original_signatures,9.0,17.0,28.0,40.0,22.0,13.0,8.0,22.0,76.0,...,0.0,3.0,24.0,58.0,44.0,23.0,1.0,0.0,0.0,0.0
457,monet_original_signatures,25.0,15.0,16.0,5.0,3.0,25.0,22.0,45.0,15.0,...,4.0,11.0,24.0,18.0,4.0,0.0,0.0,6.0,19.0,15.0


In [132]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score

X = sift_df.loc[:, 0:].values
y = sift_df[['class_name']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.5901374292643492